In [218]:
from torchtext.data import Iterator, BucketIterator
from torchtext.data import TabularDataset
from torchtext.data import Field
import torch.nn as nn
import spacy
import torch

In [219]:
tokenize = lambda x: x.split()

In [220]:
spacy_en = spacy.load("en")

In [221]:
def tokenizer(text): # create a tokenizer function
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [222]:
TEXT = Field(sequential=True, tokenize=tokenize, lower=True)
LABEL = Field(sequential=False, use_vocab=False)

In [223]:
t_datafields = [("textID", None),("text",None),("selected_text", TEXT),("sentiment", None),('negative',LABEL),('neutral',LABEL),('positive',LABEL)]
trn = TabularDataset.splits(path=".",train='train.csv',format='csv',skip_header=True,fields=t_datafields)[0]

In [224]:
len(trn)

27486

In [225]:
trn[2].selected_text

['says',
 'good',
 '(or',
 'should',
 'i',
 'say',
 'bad?)',
 'afternoon!',
 'http://plurk.com/p/wxpdj']

In [226]:
trn[0].negative

'neutral'

In [227]:
TEXT.build_vocab(trn, vectors="glove.6B.100d")
vocab = TEXT.vocab
embed = nn.Embedding(len(vocab), 100)
embed.weight.data.copy_(vocab.vectors)
vocab.vectors.size()

torch.Size([45438, 100])

In [232]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
train_iter = BucketIterator.splits(
    trn,
    sort = True,
    sort_within_batch=False,
    sort_key = lambda x: len(x.selected_text),
    batch_size = 32,
    device = device,
    repeat=True)

In [243]:
train_iter[0]